In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date

In [2]:
url = 'http://www.koeri.boun.edu.tr/scripts/lst1.asp'

r = requests.get(url)

print(r.content[:100])

b'<HTML> \r\n\r\n\r\n<meta http-equiv="Content-Type" content="text/html; charset=windows-1254">\r\n<meta http-'


In [3]:
soup = BeautifulSoup(r.content, 'html.parser')

In [4]:
pre_tag = soup.find('pre')
#if pre_tag:
    #print(pre_tag.get_text())

In [5]:
if pre_tag:
    text = pre_tag.get_text()
    lines = text.splitlines()
    data_start_index = None

    for i, line in enumerate(lines):
        if line.strip().startswith('----------'):
            data_start_index = i + 1
            break
    if data_start_index is None:
        exit()

    today_date = date.today().strftime('%Y.%m.%d')

    earthquakes = []

    for line in lines[data_start_index:]:
        if not line.strip():
            break
        parts = line.split()
        if len(parts) < 9:
            continue
        
        quake_date = parts[0]

        #Checks if earthquake happened today
        if quake_date == today_date:
            latitude_str = parts[2]
            longitude_str = parts[3]
            
            try:
                latitude = float(latitude_str)
                longitude = float(longitude_str)
            except ValueError:
                continue
            
            #Checks coordinates to see if the earthquake is close enough to affect Turkey
            #Explanation of why these coordinates where chosen is written below
            if 24.58 <= longitude <= 45 and 36 <= latitude <= 42:
                time = parts[1]
                depth = parts[4]
                MD = parts[5]
                ML = parts[6]
                Mw = parts[7]
                quality = parts[-1]
                location = " ".join(parts[8:-1])
                quake = {
                    'date': quake_date,
                    'time': time,
                    'latitude': latitude_str,
                    'longitude': longitude_str,
                    'depth_km': depth,
                    'MD': MD,
                    'ML': ML,
                    'Mw': Mw,
                    'location': location,
                    'quality': quality,
                }
                earthquakes.append(quake)
        
        elif quake_date < today_date:
            break

    #earth_list is a list to store dicts of earthquakes that have occured in the past 24h
    #earth_list is recreated each time the scraped code runs
    earth_list = []

    for q in earthquakes:
        earth_list.append(q)
        print(q)

{'date': '2025.08.11', 'time': '00:01:46', 'latitude': '39.2317', 'longitude': '28.0450', 'depth_km': '8.2', 'MD': '-.-', 'ML': '3.5', 'Mw': '-.-', 'location': 'ALAKIR-SINDIRGI (BALIKESIR)', 'quality': 'İlksel'}


The coordinates 42N, 36N, and 45E roughly gives the coordinates of Turkey and captures all major fault lines that could cause an earthquake. The coordinate 24.58E was chosen as it is the middle of the Helenic arc (center of Crete). Roughly 310km from the nearest Turkish coast. A major fault line exists near this longitude, which could pose a structural damage or tsunami risk if an earthquake larger than Mw 7.5 occurs.

In [6]:
"""
Sample data in format of l_earth for testing:

earthquake_data = [
    {'date': '2025.08.03', 'time': '13:43:07', 'latitude': '40.3658', 'longitude': '28.9940', 'depth_km': '8.1', 'MD': '-.-', 'ML': '2.8', 'Mw': '-.-'},
    {'date': '2025.08.03', 'time': '13:41:58', 'latitude': '36.2197', 'longitude': '36.2042', 'depth_km': '8.5', 'MD': '-.-', 'ML': '2.2', 'Mw': '6.2'},
    {'date': '2025.08.03', 'time': '13:11:51', 'latitude': '39.2502', 'longitude': '28.9685', 'depth_km': '11.3', 'MD': '-.-', 'ML': '1.9', 'Mw': '-.-'},
    {'date': '2025.08.03', 'time': '13:10:33', 'latitude': '39.0642', 'longitude': '25.7962', 'depth_km': '7.6', 'MD': '-.-', 'ML': '2.3', 'Mw': '7.1'},
    {'date': '2025.08.03', 'time': '12:55:01', 'latitude': '39.5011', 'longitude': '26.8837', 'depth_km': '9.8', 'MD': '-.-', 'ML': '3.5', 'Mw': '-.-'},
    {'date': '2025.08.03', 'time': '12:45:01', 'latitude': '38.5011', 'longitude': '30.8837', 'depth_km': '15.2', 'MD': '-.-', 'ML': '4.1', 'Mw': '5.9'}
]
"""
l_earth = []

for quake in earth_list:
    mw_value = quake.get('Mw')

    if mw_value and mw_value != '-.-':
        try:
            mw_magnitude = float(mw_value)

            # Check if the magnitude is greater than 6.1
            #Explanation of why 6.1 is threshold is written below
            if mw_magnitude > 6.1:
                l_earth.append(quake)

        except ValueError:
            continue

#To do: make data save to a file


The smallest deadly earthquake in Turkey in the last 50 years occurred in 1983 near Biga, with a magnitude of 6.1 (Ms). Today, the Mw (moment magnitude) scale is more commonly used instead of Ms. Another deadly earthquake in 1986 in Malatya also had a Mw of 6.1. Furthermore, the 6.1 Mw Elazığ earthquake in 2010 gave 41 casualties. This suggests that the minimum magnitude for fatal earthquakes in Turkey is 6.1 (Mw). Hence, the threshold has been set to 6.1 Mw in the code, and it is advised that this threshold remain the same for later parts of the project.